In [74]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from ipyleaflet import Map, Marker, MarkerCluster
from IPython.display import display, Markdown

<center>
    <h1>
    <strong>Project #4: Advanced Analysis</strong>
    <br/>
    </h1>
    <h5>
        <strong>Collaborators</strong>: Erik F, Rusty B, Sam L<br/><br/>
    </h5>
</center>
    <h5>
        Description: For our Project #4 we aimed to deliver an analysis describing the combative epicenters of the ongoing conflict in Ukraine, 
        then using our knowledge of these points generate locations for humanitarian efforts so they may serve the most people with minimal risk to their own facilities.
        We accomplished this with ____, _______, and _______.
    </h5>


<center><h2><strong>Data Sources</strong></h2></center>
<ul>
    <li>Source 1 Lorem ipsum dolor sit amet consectetur adipisicing elit. Debitis similique ab facilis libero quia deserunt quaerat omnis fugiat dolor eius inventore quas.</li>
    <li>Source 2 Lorem ipsum dolor sit amet consectetur adipisicing elit. Ducimus quidem ut quod, reprehenderit, nostrum mollitia sapiente error ipsum eum totam officiis.</li>
    <li>Source 3 Lorem ipsum dolor sit amet consectetur adipisicing elit. Cupiditate libero commodi nisi voluptates quod tempore tenetur facilis delectus incidunt recusandae.</li>
    <li>Source 4 Lorem ipsum dolor sit amet consectetur adipisicing elit. Dolore veniam maxime qui nobis cumque possimus ab soluta consequuntur deleniti necessitatibus.</li>
</ul>


In [75]:
#Reading and Cleaning Data
acled_data = pd.read_csv('./ACLED_DATA_PREVIOUSYEAR.csv') 
gdf = gpd.GeoDataFrame(
    acled_data, geometry=gpd.points_from_xy(acled_data.longitude, acled_data.latitude), crs="EPSG:4326"
)
filtered_gdf = gdf[gdf['fatalities']>0]
ua = gpd.read_file("./ua.json")
roads = gpd.read_file("./Ukraine_-_roads/Ukraine_-_roads.shp")
roads.to_crs("EPSG:4326")


DriverError: ./Ukraine_-_roads/Ukraine_-_roads.shp: No such file or directory

<center><h2><strong>Final Plot</strong></h2></center>

<center><h2><strong>Previous Iterations</strong></h2></center>

In [ ]:
def maximum_coverage(demand_points, facility_locations, num_facilities, service_radius):
    """
    Implements the maximum coverage algorithm.

    Args:
        demand_points: GeoDataFrame with demand points.
        facility_locations: GeoDataFrame with candidate facility locations.
        num_facilities: Number of facilities to select.
        service_radius: Service radius around each facility.

    Returns:
        GeoDataFrame of selected facilities.
    """

    selected_facilities = []
    covered_demand = []

    for _ in range(num_facilities):
        max_coverage = 0
        best_facility = None

        for _, facility in facility_locations.iterrows():
            if facility.geometry in selected_facilities:
                continue

            service_area = facility.geometry.buffer(service_radius)
            covered = demand_points[demand_points.geometry.within(service_area)]
            coverage = covered.shape[0]

            if coverage > max_coverage:
                max_coverage = coverage
                best_facility = facility

        if best_facility is not None:
            selected_facilities.append(best_facility.geometry)
            covered_demand.extend(covered_demand)

    return gpd.GeoDataFrame(geometry=selected_facilities, crs="EPSG:4326")

In [ ]:
points = []

for index, row in roads.to_crs(filtered_gdf.crs).iterrows():    
    distances = np.linspace(0, row['geometry'].length, 1)
    points.extend([row['geometry'].interpolate(distance) for distance in distances])

print("Total points: " + str(len(points)))

points_gdf = gpd.GeoDataFrame({'geometry': points})

f, ax = plt.subplots()
ax.title.set_text("1 Point per road")
points_gdf.plot(ax=ax, markersize=2, color="RED")
roads.to_crs(filtered_gdf.crs).plot(ax=ax,linewidth=0.2, edgecolor='black')

places = maximum_coverage(filtered_gdf, points_gdf, 3, .1) # .05 for the service_radius is a radius of ~5km

f, ax = plt.subplots()
ax.title.set_text("maximum coverage points")
ua.plot(ax=ax)
filtered_gdf.plot(ax=ax, color="gold", markersize=2)
places.to_crs(ua.crs).plot(ax=ax, color="red", markersize=4)

<center><h2><strong>Conclusions</strong></h2></center>

<center><h2><strong>Literature Engagements</strong></h2></center>